In [147]:
import os

BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

Config set: http://localhost:8000/v1


In [148]:
!pip3 install -q pydantic_ai openai    

In [149]:
!curl http://localhost:8000/v1/models -H "Authorization: Bearer $OPENAI_API_KEY"

{"object":"list","data":[{"id":"Qwen3-30B-A3B","object":"model","created":1752952782,"owned_by":"vllm","root":"Qwen/Qwen3-30B-A3B","parent":null,"max_model_len":40960,"permission":[{"id":"modelperm-274cf898a9c945c3ba2d8c0fc6b785fd","object":"model_permission","created":1752952782,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [150]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE

server = server = MCPServerSSE('http://localhost:8090/sse') 

In [157]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

agent_model = model = OpenAIModel("Qwen3-30B-A3B", provider=provider)

In [23]:
from pydantic_ai import Agent

time_range_agent = Agent(
    model=agent_model,
    toolsets=[server],
    system_prompt = """
    The tool to check for most optimal time frame:
    get_free_time_slots()
    Args:
    attendees (list): List of attendee identifiers (usernames, emails, or IDs)
                     whose calendars will be checked for availability.
    duration_minutes (int): Required meeting duration in minutes. Only time slots
                           with at least this much available time will be returned.
    start_date (str): Start of the search period in ISO format (YYYY-MM-DDTHH:MM:SS)
                     or similar datetime string format.
    end_date (str): End of the search period in ISO format (YYYY-MM-DDTHH:MM:SS)
                   or similar datetime string format.
 """
)

In [24]:
import asyncio
from pydantic_ai.mcp import MCPServerStdio
async def time_range_run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await time_range_agent.run(prompt)
        return result.output


In [15]:
from mcp.server.fastmcp import FastMCP
from mcp.server import Server
import json
import logging
from typing import Optional, Dict, Any
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from mcp.server.sse import SseServerTransport
import argparse
from starlette.routing import Mount, Route
from pydantic import BaseModel
from starlette.applications import Starlette
from starlette.requests import Request
import uvicorn
import os
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE

In [17]:
import time

start_time = time.time()
result = await run_async("""
    Time Frame To Check:
    'start_timeframe': '2025-07-17T00:00:00+05:30',
    'end_timeframe': '2025-07-18T11:59:59+05:30',
    'Duration (min)': 60

    User's Involved In This Analysis: userone.amd@gmail.com, usertwo.amd@gmail.com and userthree.amd@gmail.com

    Check for the most optimal timeframe slot.
""")
end_time = time.time()

execution_time = end_time - start_time
print(f"Execution time: {execution_time:.3f} seconds")
print(f"Result: {result}")

Execution time: 31.433 seconds
Result: 

The most optimal time slot available for all attendees (userone.amd@gmail.com, usertwo.amd@gmail.com, userthree.amd@gmail.com) within the specified timeframe is:

**2025-07-17 09:00:00 +0530 - 2025-07-17 16:00:00 +0530**

This 9-hour window is the only continuous period where all three attendees are free. Since the required duration is 60 minutes, any 60-minute segment within this slot (e.g., 9:00-10:00, 10:00-11:00, etc.) would work. Let me know if you'd like to narrow it further!


In [33]:
from datetime import datetime
from zoneinfo import ZoneInfo  # Python 3.9+
dt = datetime.now(ZoneInfo("Asia/Kolkata"))

In [84]:
# Define response model for meeting extraction
from pydantic import BaseModel
class MeetingTimeframe(BaseModel):
    start_timeframe: Optional[str] = None  # When to start looking for slots
    end_timeframe: Optional[str] = None    # When to stop looking for slots
    duration_minutes: Optional[int] = None  # Duration of the meeting in minutes


meeting_agent = Agent(
    model,
    output_type=MeetingTimeframe,
    system_prompt=f"""
    You are an expert at extracting meeting scheduling information from emails and messages.
    
    Current datetime: {dt.strftime("%Y-%m-%dT%H:%M:%S%z")}
    Current date: {datetime.now().strftime("%A")}
    
    Your task is to analyze the content and extract meeting scheduling parameters:
    
    1. start_timeframe: The earliest date to start looking for meeting slots
       - For "today": use current date
       - For "tomorrow": use next day
       - For day names (Monday, Tuesday, etc.): find the NEXT occurrence of that day
       - For "this week": use current date
       - For "next week": use date of next Monday
       - For specific dates: parse and use as-is
       - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    
    2. end_timeframe: The latest date to stop looking for meeting slots
       - If specific day mentioned: use same day as start_timeframe
       - If "this week" mentioned: use end of current week (Friday)
       - If "next week" mentioned: use end of next week (Friday)  
       - If range mentioned (e.g., "Monday to Wednesday"): use end of range
       - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    
    3. duration_minutes: Meeting duration in minutes
       - Parse explicit durations: "30 minutes", "1 hour", "2 hours"
       - Convert hours to minutes: 1 hour = 60 minutes
       - Common defaults: "quick chat" = 15 min, "brief meeting" = 30 min
       - If no duration specified: default to 60 minutes

    ### ✅ Few-shot Examples (Context: Saturday, 19th July 2025)
    
    #### 🗓️ Example 1
    
    **Input:** `"Let's meet Thursday"`
        
      "StartTime": "2025-07-24T00:00:00+05:30",
      "EndTime": "2025-07-24T11:59:00+05:30",
      "DurationMinutes": 60
        
    #### 🗓️ Example 2
    
    **Input:** `"30 minute call this week"`
        
      "StartTime": "2025-07-19T00:00:00+05:30",
      "EndTime": "2025-07-25T23:59:00+05:30",
      "DurationMinutes": 30
        
    #### 🗓️ Example 3
    
    **Input:** `"2 hour session tomorrow"`
    
      "StartTime": "2025-07-20T00:00:00+05:30",
      "EndTime": "2025-07-20T23:59:00+05:30",
      "DurationMinutes": 120
    
    #### 🗓️ Example 4
    
    **Input:** `"Meeting Monday to Wednesday"`
    
      "StartTime": "2025-07-21T00:00:00+05:30",
      "EndTime": "2025-07-23T23:59:00+05:30",
      "DurationMinutes": 60

    
    Be precise with date calculations and always consider the current date context.
    """,
)

In [85]:

# Define response model for meeting extraction
from pydantic import BaseModel
class AvailableSlots(BaseModel):
    start_timeframe: Optional[str] = None  # When to start the meeting

slots_agent = Agent(
    model,
    output_type=AvailableSlots,
    system_prompt=f"""
    Time Frame To Check:
    'start_timeframe': {response_data["start_timeframe"]},
    'end_timeframe': {response_data["end_timeframe"]},
    'Duration (min)': {response_data["duration_minutes"]}

    User's Involved In This Analysis: userone.amd@gmail.com, usertwo.amd@gmail.com and userthree.amd@gmail.com

    Check for the most optimal timeframe slot.
    if you get multiple timeslots pick a single optimal one and return  in specific format

    Your task is to analyze the content and extract meeting scheduling parameters:
    give only the required time slot - no extra information should be returned

    1. start_available_timeframe: Any single available timeframe from the slot
       - For specific dates: parse and use as-is
       - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    example - 
    available timezones - **2025-07-24 10:00:00 +05:30** to **2025-07-24 11:59:00 +05:30**
    return : 
    2025-07-24 10:00:00 +05:30
      """,
)

In [47]:

# Get current date for context
current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Run the agent to extract meeting information
result = await meeting_agent.run("Let's meet on Monday for 1 hour")

# Convert to dictionary for JSON serialization
response_data = {
    "start_timeframe": result.data.start_timeframe,
    "end_timeframe": result.data.end_timeframe,
    "duration_minutes": result.data.duration_minutes,
    "success": True,
    "message": "Successfully extracted meeting timeframes"
}


/tmp/ipykernel_12780/902388571.py:9: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_timeframe": result.data.start_timeframe,
/tmp/ipykernel_12780/902388571.py:10: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_timeframe": result.data.end_timeframe,
/tmp/ipykernel_12780/902388571.py:11: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": result.data.duration_minutes,


In [48]:
response_data

{'start_timeframe': '2025-07-21T00:00:00+05:30',
 'end_timeframe': '2025-07-21T23:59:00+05:30',
 'duration_minutes': 60,
 'success': True,
 'message': 'Successfully extracted meeting timeframes'}

In [49]:
current_date

'2025-07-19 17:23:16'

# MCP

In [158]:
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033c5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Project Status",
    "EmailContent": "Hi Team. Let's meet on Tuesday at 11:00 A.M and discuss about our on-going Projects."
}

In [159]:
# Define response model for meeting extraction
from pydantic import BaseModel
class MeetingTimeframe(BaseModel):
    start_timeframe: Optional[str] = None  # When to start looking for slots
    end_timeframe: Optional[str] = None    # When to stop looking for slots
    duration_minutes: Optional[int] = None  # Duration of the meeting in minutes


meeting_agent = Agent(
    model,
    output_type=MeetingTimeframe,
    system_prompt=f"""
    You are an expert at extracting meeting scheduling information from emails and messages.
    
    Current datetime: {dt.strftime("%Y-%m-%dT%H:%M:%S%z")}
    Current date: {datetime.now().strftime("%A")}
    
    Your task is to analyze the content and extract meeting scheduling parameters:
    
    1. start_timeframe: The earliest date to start looking for meeting slots
       - For "today": use current date
       - For "tomorrow": use next day
       - For day names (Monday, Tuesday, etc.): find the NEXT occurrence of that day
       - For "this week": use current date
       - For "next week": use date of next Monday
       - For specific dates: parse and use as-is
       - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    
    2. end_timeframe: The latest date to stop looking for meeting slots
       - If specific day mentioned: use same day as start_timeframe
       - If "this week" mentioned: use end of current week (Friday)
       - If "next week" mentioned: use end of next week (Friday)  
       - If range mentioned (e.g., "Monday to Wednesday"): use end of range
       - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    
    3. duration_minutes: Meeting duration in minutes
       - Parse explicit durations: "30 minutes", "1 hour", "2 hours"
       - Convert hours to minutes: 1 hour = 60 minutes
       - Common defaults: "quick chat" = 15 min, "brief meeting" = 30 min
       - If no duration specified: default to 60 minutes

    ### ✅ Few-shot Examples (Context: Saturday, 19th July 2025)
    
    #### 🗓️ Example 1
    
    **Input:** `"Let's meet Thursday"`
        
      "StartTime": "2025-07-24T00:00:00+05:30",
      "EndTime": "2025-07-24T11:59:00+05:30",
      "DurationMinutes": 60
        
    #### 🗓️ Example 2
    
    **Input:** `"30 minute call this week"`
        
      "StartTime": "2025-07-19T00:00:00+05:30",
      "EndTime": "2025-07-25T23:59:00+05:30",
      "DurationMinutes": 30
        
    #### 🗓️ Example 3
    
    **Input:** `"2 hour session tomorrow"`
    
      "StartTime": "2025-07-20T00:00:00+05:30",
      "EndTime": "2025-07-20T23:59:00+05:30",
      "DurationMinutes": 120
    
    #### 🗓️ Example 4
    
    **Input:** `"Meeting Monday to Wednesday"`
    
      "StartTime": "2025-07-21T00:00:00+05:30",
      "EndTime": "2025-07-23T23:59:00+05:30",
      "DurationMinutes": 60

    
    Be precise with date calculations and always consider the current date context.
    """,
)
# Define response model for meeting extraction
from pydantic import BaseModel
class AvailableSlots(BaseModel):
    start_timeframe: Optional[str] = None  # When to start the meeting

slots_agent = Agent(
    model,
    output_type=AvailableSlots,
    system_prompt=f"""
    Check for the most optimal timeframe slot.
    if you get multiple timeslots pick a single optimal one and return  in specific format

    Your task is to analyze the content and extract meeting scheduling parameters:
    give only the required time slot - no extra information should be returned

    If there are no available Time slots - if the tool returns Available SLOTS : []
    - you are allowed to reshedule a meeting 
    - Reshedule a meeting that is of least priority
    

    1. start_available_timeframe: Any single available timeframe from the slot
       - For specific dates: parse and use as-is
       - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    example - 
    available timezones - **2025-07-24 10:00:00 +05:30** to **2025-07-24 11:59:00 +05:30**
    return : 
    2025-07-24 10:00:00 +05:30
      """,
)

In [160]:
from datetime import datetime

def retrieve_calendar_events(user, start, end):
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

def merge_intervals(intervals):
    if not intervals:
        return []

    # Sort by start time
    intervals.sort(key=lambda x: x[0])
    merged = [intervals[0]]

    for current_start, current_end in intervals[1:]:
        last_start, last_end = merged[-1]
        if current_start <= last_end:
            merged[-1] = (last_start, max(last_end, current_end))  # Merge overlapping
        else:
            merged.append((current_start, current_end))
    return merged

import ast

def parse_list(maybe_str):
    if isinstance(maybe_str, str):
        return ast.literal_eval(maybe_str)
    return maybe_str  # it's already a list
def get_free_time_slots(attendees, duration_minutes, start_date, end_date):
    user_availabilities = []
    duration_minutes = int(duration_minutes)
    
    # Parse input dates
    start_dt = datetime.fromisoformat(start_date)
    end_dt = datetime.fromisoformat(end_date)
    
    # Input validation
    if start_dt >= end_dt:
        return "Error: Start date must be before end date"
    
    if duration_minutes <= 0:
        return "Error: Duration must be positive"
    
    attendees = parse_list(attendees)
    if not attendees:
        return "Error: No attendees provided"
    
    # Collect all busy periods
    for user in attendees:
        try:
            events = retrieve_calendar_events(user, start_date, end_date)
            for event in events:
                event_start = datetime.fromisoformat(event["StartTime"])
                event_end = datetime.fromisoformat(event["EndTime"])
                
                # Clip events to the search window
                clipped_start = max(event_start, start_dt)
                clipped_end = min(event_end, end_dt)
                
                # Only add if there's actual overlap with our time window
                if clipped_start < clipped_end:
                    user_availabilities.append((clipped_start, clipped_end))
        except Exception as e:
            print(f"Warning: Could not retrieve events for {user}: {e}")
            continue
    
    # Handle case where no events found
    if not user_availabilities:
        return f"slot 1 : {start_date} - {end_date}\n "
    
    merged_unavailable = merge_intervals(user_availabilities)
    available = []
    duration = timedelta(minutes=duration_minutes)
    
    # Check slot before first busy period
    if len(merged_unavailable) > 0:
        gap_before = merged_unavailable[0][0] - start_dt
        if gap_before >= duration:
            available.append((start_dt, merged_unavailable[0][0]))
    
    # Check gaps between busy periods
    for i in range(1, len(merged_unavailable)):
        gap_duration = merged_unavailable[i][0] - merged_unavailable[i-1][1]
        if gap_duration >= duration:
            available.append((merged_unavailable[i-1][1], merged_unavailable[i][0]))
    
    # Check slot after last busy period
    if len(merged_unavailable) > 0:
        gap_after = end_dt - merged_unavailable[-1][1]
        if gap_after >= duration:
            available.append((merged_unavailable[-1][1], end_dt))
    
    # Format output consistently (using isoformat to match your end_date parameter)
    if not available:
        return "No available time slots found for the specified duration\n "
    
    slots = ""
    idx = 1
    for slot_start, slot_end in available:
        # Use consistent isoformat for both start and end
        start_str = slot_start.isoformat()
        end_str = slot_end.isoformat()
        slots += f"slot {idx} : {start_str} - {end_str}\n "
        idx += 1
    
    return slots

In [164]:
import  time

start_time = time.time()
meeting_agent_results = meeting_agent.run(data["EmailContent"])

# Convert to dictionary for JSON serialization
response_data_meeting = {
    "start_timeframe": meeting_agent_results.data.start_timeframe,
    "end_timeframe": meeting_agent_results.data.end_timeframe,
    "duration_minutes": meeting_agent_results.data.duration_minutes,
    "success": True,
    "message": "Successfully extracted meeting timeframes"
}

attendees = []
for attendee in data["Attendees"]:
    attendees.append(attendee["email"])

slots = get_free_time_slots(attendees, duration_minutes = response_data_meeting["duration_minutes"], start_date = response_data_meeting["start_timeframe"], end_date=response_data_meeting["end_timeframe"])

slot_prompt = f"""

    Available SLOTS: [{slots}] ,

"""
slots_agent_results = await slots_agent.run(slot_prompt)

# Convert to dictionary for JSON serialization
response_data_slots = {
    "start_timeframe": slots_agent_results.data.start_timeframe
}


raw = result.strip()
dt = datetime.strptime(raw, "%Y-%m-%d %H:%M:%S %z")
available_start_time = dt.strftime("%Y-%m-%dT%H:%M:%S%z")
available_end_dt = dt + timedelta(minutes=response_data_meeting["duration_minutes"])
available_end_time = available_end_dt.strftime("%Y-%m-%dT%H:%M:%S%z")



output = {}
output["Request_id"] = data["Request_id"]
output["Datetime"] = data["Datetime"]
output["Location"] = data["Location"]
output["Request_id"] = data["Request_id"]
output["From"] = data["From"]
output["Location"] = data["Location"]
output["Attendees"] = data["Attendees"]
output["Subject"] = data["Subject"]
output["EmailContent"] = data["EmailContent"]
output["EventStart"] = available_start_time
output["EventEnd"] = available_end_time
output["Duration_mins"] =  str(response_data_meeting["duration_minutes"])

output["MetaData"] = {}


for i  in range(len(output["Attendees"])):
    email = output["Attendees"][i]["email"]
    output["Attendees"][i]["events"] = retrieve_calendar_events(
        user = email,
        start = response_data_meeting["start_timeframe"], 
        end = response_data_meeting["end_timeframe"]
    )


end_time = time.time()

execution_time = end_time - start_time
print(f"Execution time: {execution_time:.3f} seconds")
print(output)

AttributeError: 'coroutine' object has no attribute 'data'

In [162]:
output

{'Request_id': '6118b54f-907b-4451-8d48-dd13d76033c5',
 'Datetime': '19-07-2025T12:34:55',
 'Location': 'IISc Bangalore',
 'From': 'userone.amd@gmail.com',
 'Attendees': [{'email': 'usertwo.amd@gmail.com',
   'events': [{'StartTime': '2025-07-22T09:00:00+05:30',
     'EndTime': '2025-07-22T17:00:00+05:30',
     'NumAttendees': 1,
     'Attendees': ['SELF'],
     'Summary': 'Workshop - AI Agents'}]},
  {'email': 'userthree.amd@gmail.com',
   'events': [{'StartTime': '2025-07-22T09:00:00+05:30',
     'EndTime': '2025-07-22T17:00:00+05:30',
     'NumAttendees': 1,
     'Attendees': ['SELF'],
     'Summary': 'Workshop - AI Agents'}]}],
 'Subject': 'Project Status',
 'EmailContent': "Hi Team. Let's meet on Tuesday at 11:00 A.M and discuss about our on-going Projects.",
 'EventStart': '2025-07-24T10:00:00+0530',
 'EventEnd': '2025-07-24T11:00:00+0530',
 'Duration_mins': '60',
 'MetaData': {}}

In [163]:
response_data_meeting

{'start_timeframe': '2025-07-22T11:00:00+05:30',
 'end_timeframe': '2025-07-22T11:59:00+05:30',
 'duration_minutes': 60,
 'success': True,
 'message': 'Successfully extracted meeting timeframes'}